In [1]:
#!/usr/bin/env python
# # -*- coding: utf-8 -*-
#!/usr/bin/env python
#!pip install tensorflow==1.15

import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob, os, inspect
import argparse
import time
import seaborn as sns
# import matplotlib
# matplotlib.use("Qt5Agg")
# #matplotlib.use('Agg')
# from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt
import pandas as pd
import sys,inspect
sys.path.append("C:\\Users\\Vinamr J\\Documents\\qic\\qrc\\hqr\\hqr-master\\chaos\\Postprocess")
sys.path.append("C:\\Users\\Vinamr J\\Documents\\qic\\qrc\\hqr\\hqr-master\\chaos\\Methods\\Models\\Utils")

import utils  # for utils.py
from utils import *
import global_utils  # for global_utils.py

# import utils
# from utils import *
# import sys
# sys.path.append('../Methods/Models/Utils')
# import global_utils

In [2]:
sysname, tidx = "SST", 2
file_path = os.path.abspath("GRU_analysis.ipynb")
current_directory = os.path.dirname(file_path)
mid_directory = os.path.dirname(current_directory)
parent_directory = os.path.dirname(mid_directory)

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

#current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
results_dir = os.path.join(os.path.dirname(mid_directory), "Results")  # Use os.path.join for path concatenation
print(results_dir)

eval_path = os.path.join(results_dir, sysname, 'Evaluation_Data')
print(eval_path)

model_path = os.path.join(results_dir, sysname, 'Trained_Models')
print(model_path)

fig_path = os.path.join(results_dir, sysname, 'Eval_Figures')
if not os.path.isdir(fig_path):
    os.makedirs(fig_path)  # Use os.makedirs to create parent directories if needed
print(fig_path)

Current Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Data\SST
Parent Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Evaluation_Data
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Trained_Models
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Eval_Figures


In [10]:
models = []
samples = []
size_values = [80,120,250,500,1000,2000]
layer_values = [1,2,3]
for size in size_values:
    for layers in layer_values:
        string1 = f"RNN-gru-RDIM_5-N_used_427-NLAY_{layers}-SLAY_{size}-ISH_statefull-SL_16-PL_16-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3"
        string2 = f"GRU-{size}-({layers})"
        models.append([os.path.join(eval_path,string1),string2])
        samples.append(string2)
title = "Sea Surface Temperatures"

In [3]:
# models = [[os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_20-IUL_0-REG_1e-07-AU_0-NICS_3"),"HQR-5,V-10,DL=40,IPL=20,n_tests=3"],
#           [os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_8-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_8-IUL_0-REG_1e-07-AU_0-NICS_1"),"HQR-5,V-10,DL=8,IPL=8,n_tests=1"],
#           [os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_1"),"HQR-5,V-10,DL=40,IPL=300,n_tests=1"]]
# samples = ["HQR-5,V-10,DL=40,IPL=20,n_tests=3","HQR-5,V-10,DL=8,IPL=8,n_tests=1","HQR-5,V-10,DL=40,IPL=300,n_tests=1"]
# title = "Sea Surface Temperatures"

In [11]:
rmse_dict = dict()
targets = dict()
outputs = dict()
sp_outputs = dict()
sp_targets = dict()
Wouts = dict()
coeffs = dict()
rmnse_avg_train_dict = dict()
rmnse_avg_test_dict = dict()
for i in range(len(models)):
    rfolder, label = models[i][0], models[i][1]
    fname = os.path.join(rfolder, 'results.pickle')
    if os.path.isfile(fname):
        with open(fname, 'rb') as rfile:
            try:
                rs = pickle.load(rfile)
            except:
                continue
            #print(rs.keys())
            qs = QResults()
            qs.rmnse_avg_test = rs['rmnse_avg_TEST']
            qs.rmnse_avg_train = rs['rmnse_avg_TRAIN']
            qs.n_pred_005_avg_test = rs['num_accurate_pred_005_avg_TEST']
            qs.n_pred_005_avg_train = rs['num_accurate_pred_005_avg_TRAIN']
            qs.n_pred_050_avg_test = rs['num_accurate_pred_050_avg_TEST']
            qs.n_pred_050_avg_train = rs['num_accurate_pred_050_avg_TRAIN']
            qs.model_name = rs['model_name']
            #if qs.rmnse_avg_test != np.inf and qs.rmnse_avg_train != np.inf:
                #print(rs.keys())
            #print(qs.model_name)
            #print('train={}, test={}'.format(qs.rmnse_avg_train, qs.rmnse_avg_test))
            #qs.info()
            rmnse_avg_train_dict[label] = rs['rmnse_avg_TRAIN']
            rmnse_avg_test_dict[label] = rs['rmnse_avg_TEST']
            pred_test = rs['predictions_all_TEST']
            truth_test = rs['truths_all_TEST']
            print(pred_test.shape)
            M = len(pred_test)
            print('{} Number of test'.format(qs.model_name), M)
            rmsels = []
            for j in range(M):
                    rmsels.append(calNRMSE(pred_test[j], truth_test[j]))

            rmse_dict[label] = np.mean(np.array(rmsels), axis=0)
            print(rmse_dict[label].shape)


            targets[label] = truth_test[0]
            outputs[label] = pred_test[0]

            # For frequency
            sp_outputs[label] = rs['sp_pred_TEST']
            sp_targets[label] = rs['sp_true_TEST']
    else:
        print('Not found {}'.format(fname))
    # if trained_models is not None:
    #     rfolder, label = trained_models[i][0], trained_models[i][1]
    #     fname = os.path.join(rfolder, 'data.pickle')
    #     if os.path.isfile(fname):
    #         print('File existed: ', fname)
    #         with open(fname, 'rb') as rfile:
    #             #try:
    #             rs = pickle.load(rfile)
    #             #except:
    #             #    continue
    #             print(rs.keys())
    #             coeffs[label] = np.array(rs['coeffs'])
    #             Wouts[label]  = rs['W_out'][:-1].reshape((coeffs[label].shape[0], -1))
    #             print(Wouts[label].shape, coeffs[label].shape)
    #     else:
    #         print('Not found saved model {}'.format(fname))

(3, 300, 5)
RNN-gru-RDIM_5-N_used_427-NLAY_1-SLAY_80-ISH_statefull-SL_16-PL_16-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-gru-RDIM_5-N_used_427-NLAY_2-SLAY_80-ISH_statefull-SL_16-PL_16-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-gru-RDIM_5-N_used_427-NLAY_3-SLAY_80-ISH_statefull-SL_16-PL_16-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-gru-RDIM_5-N_used_427-NLAY_1-SLAY_120-ISH_statefull-SL_16-PL_16-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-gru-RDIM_5-N_used_427-NLAY_2-SLAY_120-ISH_statefull-SL_16-PL_16-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-gru-RDIM_5-N_used_427-NLAY_3-SLAY_120-ISH_statefull-SL_16-PL_16-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-gru-RDIM_5-N_used_427-NLAY_1-SLAY_250-ISH_sta

In [12]:
model_test_error = []
test_avg_error = []
for label in samples:
    model_test_error.append([label,np.mean(rmse_dict[label])])
    test_avg_error.append(np.mean(rmse_dict[label]))
print(model_test_error)
print("Test Average error = ",np.mean(test_avg_error))


[['GRU-80-(1)', 1.2518141803772236], ['GRU-80-(2)', 1.3374489502864877], ['GRU-80-(3)', 1.4517902070546562], ['GRU-120-(1)', 1.521963977644555], ['GRU-120-(2)', 1.7518134425522949], ['GRU-120-(3)', 1.3332959537400826], ['GRU-250-(1)', 1.502475857437001], ['GRU-250-(2)', 1.2604286994907787], ['GRU-250-(3)', 1.4144465052215447], ['GRU-500-(1)', 1.123836246670689], ['GRU-500-(2)', 1.0800059847354848], ['GRU-500-(3)', 1.0799142959096673], ['GRU-1000-(1)', 1.4360312915261375], ['GRU-1000-(2)', 1.435279412291051], ['GRU-1000-(3)', 1.6224434751344832], ['GRU-2000-(1)', 1.2188414706001025], ['GRU-2000-(2)', 1.2570944052063695], ['GRU-2000-(3)', 1.3748800274277608]]
Test Average error =  1.358544687961465


In [13]:
print(rmnse_avg_train_dict)
print(rmnse_avg_test_dict)
error_array_test = []
error_array_train = []
for label in samples:
    error_array_train.append(rmnse_avg_train_dict[label])
    error_array_test.append(rmnse_avg_test_dict[label])
print("Train average error for GRU =", np.mean(error_array_train))
print("Test average error for GRU =", np.mean(error_array_test))

{'GRU-80-(1)': 1.1308661484806573, 'GRU-80-(2)': 1.0553558737013635, 'GRU-80-(3)': 1.1187798424153286, 'GRU-120-(1)': 1.1383827980663197, 'GRU-120-(2)': 1.2301409008030952, 'GRU-120-(3)': 1.1837960692929093, 'GRU-250-(1)': 1.2485249052191367, 'GRU-250-(2)': 1.0297682935108483, 'GRU-250-(3)': 1.1442720416338792, 'GRU-500-(1)': 1.1914072438257466, 'GRU-500-(2)': 1.169359890276861, 'GRU-500-(3)': 1.21899482423078, 'GRU-1000-(1)': 1.0820753830826781, 'GRU-1000-(2)': 1.0615887912188895, 'GRU-1000-(3)': 1.1830373783687658, 'GRU-2000-(1)': 1.0577876736113436, 'GRU-2000-(2)': 1.1137049589159553, 'GRU-2000-(3)': 1.0457178494947885}
{'GRU-80-(1)': 1.113429916230089, 'GRU-80-(2)': 1.190891151010446, 'GRU-80-(3)': 1.245566244586139, 'GRU-120-(1)': 1.3050297178053445, 'GRU-120-(2)': 1.4699487030270206, 'GRU-120-(3)': 1.2234758625723634, 'GRU-250-(1)': 1.3846936193523007, 'GRU-250-(2)': 1.1156727686897587, 'GRU-250-(3)': 1.2312905101065743, 'GRU-500-(1)': 1.020738938638129, 'GRU-500-(2)': 0.96386130

In [14]:
models = []
samples = []
size_values = [80,120,250,500,1000,2000]
layer_values = [1,2,3]
for size in size_values:
    for layers in layer_values:
        string1 = f"RNN-gru-RDIM_5-N_used_427-NLAY_{layers}-SLAY_{size}-ISH_statefull-SL_16-PL_16-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3"
        string2 = f"GRU-{size}-({layers})"
        models.append([os.path.join(model_path,string1),string2])
        samples.append(string2)
title = "Sea Surface Temperatures"

In [15]:
train_time_dict = dict()
memory_dict = dict()
trainable_parameters_dict = dict()
for i in range(len(models)):
    rfolder, label = models[i][0], models[i][1]
    fname = os.path.join(rfolder, 'data.pickle')
    if os.path.isfile(fname):
        with open(fname, 'rb') as rfile:
            try:
                rs = pickle.load(rfile)
            except:
                continue
            #print(rs.keys())
            train_time_dict[label]=rs["total_training_time"]
            trainable_parameters_dict[label]=rs["n_trainable_parameters"]
            memory_dict[label]=rs["memory"]
    else:
        print('Not found {}'.format(fname))

In [16]:
train_time_dict

{'GRU-80-(1)': 1.5055692195892334,
 'GRU-80-(2)': 2.1024346351623535,
 'GRU-80-(3)': 3.0245089530944824,
 'GRU-120-(1)': 2.5037057399749756,
 'GRU-120-(2)': 3.9901349544525146,
 'GRU-120-(3)': 6.4084131717681885,
 'GRU-250-(1)': 1.5488948822021484,
 'GRU-250-(2)': 2.3246452808380127,
 'GRU-250-(3)': 3.1490418910980225,
 'GRU-500-(1)': 1.5599327087402344,
 'GRU-500-(2)': 2.845921039581299,
 'GRU-500-(3)': 4.146128177642822,
 'GRU-1000-(1)': 1.8387813568115234,
 'GRU-1000-(2)': 4.367665529251099,
 'GRU-1000-(3)': 7.139265775680542,
 'GRU-2000-(1)': 3.526146411895752,
 'GRU-2000-(2)': 8.62413239479065,
 'GRU-2000-(3)': 14.147461652755737}

In [17]:
memory_dict

{'GRU-80-(1)': 331.375,
 'GRU-80-(2)': 347.16796875,
 'GRU-80-(3)': 359.87109375,
 'GRU-120-(1)': 332.671875,
 'GRU-120-(2)': 350.7109375,
 'GRU-120-(3)': 367.1484375,
 'GRU-250-(1)': 337.44921875,
 'GRU-250-(2)': 364.1875,
 'GRU-250-(3)': 391.46484375,
 'GRU-500-(1)': 356.0234375,
 'GRU-500-(2)': 422.44921875,
 'GRU-500-(3)': 481.01953125,
 'GRU-1000-(1)': 428.98046875,
 'GRU-1000-(2)': 624.84375,
 'GRU-1000-(3)': 822.03125,
 'GRU-2000-(1)': 701.5625,
 'GRU-2000-(2)': 1447.6484375,
 'GRU-2000-(3)': 2224.44140625}

In [18]:
trainable_parameters_dict

{'GRU-80-(1)': 21045,
 'GRU-80-(2)': 59685,
 'GRU-80-(3)': 98325,
 'GRU-120-(1)': 45965,
 'GRU-120-(2)': 132725,
 'GRU-120-(3)': 219485,
 'GRU-250-(1)': 193255,
 'GRU-250-(2)': 569005,
 'GRU-250-(3)': 944755,
 'GRU-500-(1)': 761505,
 'GRU-500-(2)': 2263005,
 'GRU-500-(3)': 3764505,
 'GRU-1000-(1)': 3023005,
 'GRU-1000-(2)': 9026005,
 'GRU-1000-(3)': 15029005,
 'GRU-2000-(1)': 12046005,
 'GRU-2000-(2)': 36052005,
 'GRU-2000-(3)': 60058005}